In [ ]:
# Motivated by https://github.com/lukasheinrich/lhoodbinder2/blob/rootless/PlotContour.ipynb

In [ ]:
%matplotlib widget

import array
import json

import matplotlib.pyplot as plt
import mplhep
import numpy as np
from descartes import PolygonPatch
from shapely.geometry.polygon import Polygon

In [ ]:
# Find way to simplify and remove these
# from plotting import harvest_results, make_plot
from plotting import make_plot

In [ ]:
with open("results.json") as read_file:
    _results = json.load(read_file)

with open("harvests.json") as read_file:
    _harvests = json.load(read_file)

In [ ]:
# load atlas style
plt.style.use(mplhep.style.ATLAS)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(9.33, 7)

ax.set_xlabel(r"$m(\tilde{\chi}_{1}^{\pm}/\tilde{\chi}_{2}^{0})$ [GeV]")
ax.set_ylabel(r"$m(\tilde{\chi}_{1}^{0})$ [GeV]")

In [ ]:
# Set plot ranges
mass_ranges = np.asarray(
    [values["mass_hypotheses"] for _, values in _results.items()]
).T

ax.set_xlim(mass_ranges[0].min(), mass_ranges[0].max() + 100)
ax.set_ylim(mass_ranges[1].min(), mass_ranges[1].max() + 100);

In [ ]:
# show the mass hypotheses being interpolated between
ax.scatter(*mass_ranges, s=20, alpha=0.2);

In [ ]:
# make_plot(ax,label = 'Open Likelihood (in progress)', color = 'steelblue', showPoints = True)

In [ ]:
_harvests["C1N2_Wh_hbb_1000_0"]